In [1]:
# Загружаем библиотеки
import pandas as pd # Датафреймы pandas
from sklearn.feature_extraction.text import TfidfVectorizer # Модель TF-IDF
from sklearn.ensemble import RandomForestRegressor # Модель случайного леса
from sklearn.model_selection import train_test_split # Разделение выборки на обучающую и тестовую
from sklearn.metrics import mean_squared_error # Средняя квадратичная ошибка
import numpy as np # Математика

#### Загрузка данных

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

#### Обработка данных

In [3]:
# В списке жанров заменим разделители на пробелы, удалим тире, преобразуем к нижнему регистру
movies['genres'] = movies['genres'].apply(lambda string: ' '.join(string.replace('-', '').split('|')).lower())

In [4]:
# Берём столбцы с номерами пользователей, фильмов и теги, объединяем с таблицей фильмов по номеру фильма
movies_tagged = movies.join(tags[['userId', 'movieId', 'tag']].set_index('movieId'), on = 'movieId')
# Преобразуем теги: удалим все пробелы, приведём к нижнему регистру
movies_tagged['tag'] = movies_tagged['tag'].apply(lambda string: str(string).replace('r:', '').replace('(', '').replace(')', '').replace('-', '').lower())

In [5]:
# Дополняем список фильмов с тегами и дополняем оценками пользователей
movies_tagged_raitings = movies_tagged.join(ratings[['userId', 'movieId', 'rating']].set_index(['movieId', 'userId']), on = ['movieId', 'userId'])
# Удалим строки с пустыми значениями
movies_tagged_raitings.dropna(inplace = True)

#### Расчёт средних рейтингов

In [6]:
# Создадим столбцы для среднего рейтинга фильма и среднего рейтинга пользователя
movies_tagged_raitings['mean_movie_rating'] = 0
movies_tagged_raitings['mean_user_rating'] = 0
# Переставим столбцы местами для удобства просмотра
movies_tagged_raitings = movies_tagged_raitings[['movieId', 'mean_movie_rating','title', 'userId', 'mean_user_rating', 'rating', 'genres', 'tag']]
# Сбросим индекс
movies_tagged_raitings = movies_tagged_raitings.reset_index(drop=True)

In [7]:
# Средний рейтинг фильма по всем пользователям
movies_tagged_raitings['mean_movie_rating'] = movies_tagged_raitings['movieId'].apply(lambda movieId: movies_tagged_raitings[movies_tagged_raitings['movieId'] == movieId]['rating'].mean())

In [8]:
# Средний рейтинг пользователя по всем фильмам
movies_tagged_raitings['mean_user_rating'] = movies_tagged_raitings['userId'].apply(lambda userId: movies_tagged_raitings[movies_tagged_raitings['userId'] == userId]['rating'].mean())

#### Модель TF-IDF

In [9]:
# Вызываем модель TF-IDF и обучаем её, преобразуем к матрице
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(list(movies_tagged_raitings['genres'] + ' ' + movies_tagged_raitings['tag']))

#### Модель случайного леса и оценка точности

In [10]:
# Разделяем выборку на обучающую и тестовую
all_data = np.column_stack([tfidf_matrix.toarray(), movies_tagged_raitings['mean_movie_rating'].to_numpy(), movies_tagged_raitings['mean_user_rating'].to_numpy()])
X_train, X_test, y_train, y_test = train_test_split(all_data[:500], movies_tagged_raitings['rating'][:500], random_state = 42)

In [11]:
# Используем модель случайного леса
model = RandomForestRegressor(random_state = 42)
# Обучаем модель на обучающей выборке
model.fit(X_train, y_train)
print(f'- показатель точности {model.score(X_test, y_test)}')
print(f'- RMSE {mean_squared_error(y_test, model.predict(X_test), squared = False)}')

- показатель точности 0.8982008032128515
- RMSE 0.3184211048281819
